In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


sns.set_style("whitegrid")
sns.set_context("poster", font_scale=0.75)
sns.set_palette("colorblind")


In [ ]:
data = pd.read_csv("out/frag.csv")
buckets = data.columns[2:]
data = data[["i", *buckets]]
fix, ax = plt.subplots()
fix.set_figwidth(10)
fix.set_figheight(6)
data.plot.area(ax=ax, x="i", xlim=(0, 100), xlabel="iteration",
               ylabel="amount of tables", legend="reverse")
handles, labels = ax.get_legend_handles_labels()
ax.legend(reversed(handles), reversed(labels), title="free pages", loc="right")
